In [122]:
import numpy as np
import pandas as pd
import random as rd

N = 10000
FEATURES = 10

cols = "abcdefghijkmnopqrstuv"
columns = list(cols)[:FEATURES]

x = np.random.rand(N, FEATURES)

df = pd.DataFrame(x, columns = columns)
df["y"] = np.sin(df["a"].values) + np.cos(df["b"].values) + np.random.rand(N) * 0.001

df.to_csv("data.csv")


unary_funs = ["sinf", "cosf", "sqrtf"]
operators = ["+", "-"]

def random_program(depth=4):
    r = rd.randint(0,100)
    if depth == 0 or r < 30:
        c = rd.choice(columns)
        return f"_{c}_"
    elif r < 80:
        c = rd.choice(unary_funs)
        r = random_program(depth-1)
        return f"{c}({r})"
    else:
        c = rd.choice(operators)
        r1 = random_program(depth-1)
        r2 = random_program(depth-1)
        return f"({r1}) {c} ({r2})"


with open("functions.txt", "w") as f:
    for _ in range(1000):
        f.write(random_program() + "\n")



In [123]:
import time

start_time = time.time()

df = pd.read_csv("data.csv")

funs = [ line.strip() for line in open("functions.txt").readlines() ]



def score(line):
    for u in ["sinf", "cosf", "tanf", "sqrtf", "expf"]:
        line = line.replace(u, f"np.{u[:-1]}")
    for c in df.columns:
        line = line.replace(f"_{c}_", f"(df[\"{c}\"].values)")
    a = eval(line)
    b = df["y"]
    e = np.square(np.subtract(a, b)).mean()
    return e

l = funs[0]
print(score(l), l)

r = min([ (score(line), line) for line in funs ])
print(f"{r[0]} {r[1]}")

end_time = time.time()
execution_time = end_time - start_time
print(f"Execution time: {execution_time} seconds")

<string>:1: RuntimeWarning: invalid value encountered in sqrt


0.7062157792152494 sqrtf(cosf((cosf(_a_)) + (_c_)))
0.08516956572096562 sqrtf((cosf(_i_)) + (cosf(sinf(_j_))))
Execution time: 16.45693588256836 seconds


In [1]:
!pip install pycuda

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 6.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 11.9 MB/s eta 0:00:00
  Created wheel for pycuda: filename=pycuda-2023.1-cp310-cp310-linux_x86_64.whl size=661205 sha256=ff4681f2e9d4e8db2fdd8ad969fa9d71a985a28bc3ca13986100b30a1eb8d503
  Stored in directory: /root/.cache/pip/wheels/46/65/06/b997165edd2fd9690c3497ca54ea4485b571d7bd959c21c6c4
Successfully built pycuda


In [127]:
import pandas as pd
import numpy as np
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule

start_time = time.time()

# Load DataFrame and target column
df = pd.read_csv('data.csv')
b = df['y'].values.astype(np.float32)
block_size = 256

# Read functions from file
funs = [line.strip() for line in open('functions.txt').readlines()]

# CUDA Kernel for MSE calculation
mse_kernel_code = """
__global__ void mse_kernel(float *a, float *b, float *mse, int n) {
    int idx = threadIdx.x + blockIdx.x * blockDim.x;
    if (idx < n) {
        float diff = a[idx] - b[idx];
        mse[idx] = diff * diff;
    }
}

__global__ void reduce_min(float *input, float *output, int n) {
    extern __shared__ float sdata[];
    unsigned int tid = threadIdx.x;
    unsigned int i = blockIdx.x * blockDim.x + threadIdx.x;
    if (i < n) {
        sdata[tid] = input[i];
    } else {
        sdata[tid] = 1000000;
    }
    __syncthreads();

    for (unsigned int s = blockDim.x / 2; s > 0; s >>= 1) {
        if (tid < s) {
            sdata[tid] = min(sdata[tid], sdata[tid + s]);
        }
        __syncthreads();
    }

    if (tid == 0) output[blockIdx.x] = sdata[0];
}
"""

# Prepare and compile the CUDA kernel

mse_kernel_module = SourceModule(mse_kernel_code)
mse_kernel_function = mse_kernel_module.get_function("mse_kernel")

# List to store MSEs
mse_list = []
b_gpu = cuda.mem_alloc(b.nbytes)
cuda.memcpy_htod(b_gpu, b)

for line in funs:

    for u in ["sinf", "cosf", "tanf", "sqrtf", "expf"]:
        line = line.replace(u, f"np.{u[:-1]}")
    for c in df.columns:
        line = line.replace(f"_{c}_", f"df['{c}'].values")

    # Evaluate the expression
    a = eval(line)
    n = a.size
    grid_size = int(np.ceil(n / block_size))

    a = a.astype(np.float32)

    a_gpu = cuda.mem_alloc(a.nbytes)

    mse_gpu = cuda.mem_alloc(a.nbytes)

    cuda.memcpy_htod(a_gpu, a)

    mse_result = np.empty_like(a)

    # Run kernel
    mse_kernel_function(a_gpu, b_gpu, mse_gpu, np.int32(n), block=(block_size, 1, 1), grid=(grid_size, 1))

    # Copy the result back to host
    cuda.memcpy_dtoh(mse_result, mse_gpu)

    # Calculate mean of MSE and append to list
    mse_list.append(mse_result.mean())

    if len(mse_list) == 1:
      first_mse = mse_list[0]
      first_function = funs[0]
      print(first_mse, first_function)

    a_gpu.free()
    mse_gpu.free()

######################################################################################################################################################################
b_gpu.free()

mse_kernel_function = mse_kernel_module.get_function("reduce_min")

mse_array = np.array(mse_list, dtype=np.float32)
mse_gpu = cuda.mem_alloc(mse_array.nbytes)
cuda.memcpy_htod(mse_gpu, mse_array)

min_mse_gpu = cuda.mem_alloc(mse_array.nbytes)

num_blocks = int(np.ceil(len(mse_list) / block_size))

mse_kernel_function(mse_gpu, min_mse_gpu, np.int32(len(mse_list)), block=(block_size, 1, 1), grid=(num_blocks, 1), shared=block_size * 4)


min_mse_result = np.empty(num_blocks, dtype=np.float32)
cuda.memcpy_dtoh(min_mse_result, min_mse_gpu)


min_mse = np.min(min_mse_result)
best_function = funs[mse_list.index(min_mse)]
print(f"Best MSE: {min_mse}, Function: {best_function}")

mse_gpu.free()
min_mse_gpu.free()

end_time = time.time()
execution_time = end_time - start_time
print(f"Execution time: {execution_time} seconds")


<string>:1: RuntimeWarning: invalid value encountered in sqrt
/usr/local/lib/python3.10/dist-packages/google/colab/_variable_inspector.py:27: UserWarning: module in out-of-thread context could not be cleaned up
  globals().clear()


nan sqrtf(cosf((cosf(_a_)) + (_c_)))
Best MSE: 0.08516955375671387, Function: sqrtf((cosf(_i_)) + (cosf(sinf(_j_))))
Execution time: 13.653202056884766 seconds
